<a href="https://colab.research.google.com/github/jdasam/ant5015/blob/main/notebooks/10th_week_automatic_music_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Download Dataset

In [ ]:
!gdown --id 1L99FSzloXGQsmc-OwsVKggvQNF9niN2M
!unzip -q maestro_small.zip


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/teo/.local/lib/python3.8/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L99FSzloXGQsmc-OwsVKggvQNF9niN2M
From (redirected): https://drive.google.com/uc?id=1L99FSzloXGQsmc-OwsVKggvQNF9niN2M&confirm=t&uuid=98248691-46b9-4266-8b7f-e4d57278c348
To: /home/teo/userdata/git_libraries/ant5015-2024/notebooks/maestro_small.zip
  3%|█▏                                    | 37.2M/1.14G [00:02<00:52, 20.8MB/s]^C
Traceback (most recent call last):
  File "/home/teo/.local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/home/teo/.local/lib/python3.8/s

In [ ]:
!pip install mido pretty_midi

## 1. Preprocess Data

In [ ]:
from mido import Message, MidiFile, MidiTrack, MetaMessage


## 2. Model

## 3. Training